In [23]:
import geopandas as gpd

## Open GIS Data

In [24]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)
states_gdf = states_gdf[["STUSPS", "geometry"]]

## Open Dollar General Data

In [29]:
dollar_generals_gdf = gpd.read_file("data/dollar_generals_per_state.gpkg")
dollar_generals_gdf = dollar_generals_gdf[["STUSPS", "DOLLAR_GENERALS"]]

## Open Family Dollar Data

In [30]:
family_dollar_gdf = gpd.read_file("data/family_dollar_per_state.gpkg")
family_dollar_gdf = family_dollar_gdf[["STUSPS", "FAMILY_DOLLARS"]]

## Open Dollar Tree Data

In [31]:
dollar_tree_gdf = gpd.read_file("data/dollar_trees_per_state.gpkg")
dollar_tree_gdf = dollar_tree_gdf[["STUSPS", "DOLLAR_TREE"]]

## Merge Data

In [35]:
dg_states_gdf = states_gdf.merge(dollar_generals_gdf, on="STUSPS", how="left")
fd_dg_states_gdf = dg_states_gdf.merge(family_dollar_gdf, on="STUSPS", how="left")
fd_dg_dt_gdf = fd_dg_states_gdf.merge(dollar_tree_gdf, on="STUSPS", how="left")

In [38]:
fd_dg_dt_gdf = fd_dg_dt_gdf.fillna(0)

In [39]:
fd_dg_dt_gdf["DOLLAR_GENERALS"] = fd_dg_dt_gdf["DOLLAR_GENERALS"].astype(int)
fd_dg_dt_gdf["FAMILY_DOLLARS"] = fd_dg_dt_gdf["FAMILY_DOLLARS"].astype(int)
fd_dg_dt_gdf["DOLLAR_TREE"] = fd_dg_dt_gdf["DOLLAR_TREE"].astype(int)

In [50]:
def more_stores(row) -> str:
    dg = row["DOLLAR_GENERALS"]
    fd = row["FAMILY_DOLLARS"]
    dt = row["DOLLAR_TREE"]
    if (dg > fd) and (dg > dt):
        return "More Dollar Generals"
    if (fd > dg) and (fd > dt):
        return "More Family Dollars"
    if (dt > dg) and (dt > fd):
        return "More Dollar Trees"
    if dt == 0 and dg == 0 and dt == 0:
        return "No Locations"

    # For DC
    if fd == dt and dg != dt:
        return "Same Amount of FD and DT"

In [52]:
fd_dg_dt_gdf["label"] = fd_dg_dt_gdf.apply(lambda row: more_stores(row), axis=1)

In [53]:
fd_dg_dt_gdf = fd_dg_dt_gdf.to_crs(9311)
fd_dg_dt_gdf.to_file("data/fd_dg_dt.gpkg")